# Install TensorRt and Torch2TRT on Google Colab. You have to restart the notebook for change to take effect. Ignore this part on Jetbot.

### Install TensorRt
1. Create new folder 'TRT' on Google drive
2. Download deb package for cuda 10 and place into TRT folder
3. Run script below

### Download 
https://developer.nvidia.com/compute/machine-learning/tensorrt/secure/7.0/7.0.0.11/local_repo/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216_1-1_amd64.deb and place it in your google drive under a folder "TRT".

In [ ]:
import os
os.environ["os1"]="ubuntu1804"
os.environ["tag"]="cuda10.0-trt7.0.0.11-ga-20191216"
os.environ["version"]="7.0.0-1+cuda10.0"
os.chdir('/content/drive/MyDrive/TRT/')
!sudo dpkg -i nv-tensorrt-repo-${os1}-${tag}_1-1_amd64.deb
!sudo apt-key add /var/nv-tensorrt-repo-${tag}/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install libnvinfer7=${version} libnvonnxparsers7=${version} libnvparsers7=${version} libnvinfer-plugin7=${version} libnvinfer-dev=${version} libnvonnxparsers-dev=${version} libnvparsers-dev=${version} libnvinfer-plugin-dev=${version} python-libnvinfer=${version} python3-libnvinfer=${version}
!sudo apt-mark hold libnvinfer7 libnvonnxparsers7 libnvparsers7 libnvinfer-plugin7 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python-libnvinfer python3-libnvinfer
!sudo apt-get install tensorrt=${version}
!sudo apt-get install python3-libnvinfer-dev=${version}

### Install torch2trt
1. Create folder 'torch2trt' on Google drive
2. Run script below
3. Restart the notebook after installation success

In [ ]:
!git clone https://github.com/NVIDIA-AI-IOT/torch2trt /drive/MyDrive/torch2trt
! cd /drive/MyDrive/torch2trt && pwd && python setup.py install

# Convert PyTorch model using torch2trt

In [ ]:
import os
import time

# Helper function to get size
def get_size(file_path):
    b = os.path.getsize(file_path)
    print("File size in byte: ", b)

# Helper function to measure inference time and return model predictions
def run_inference(model, data):
    t0 = time.time()
    output = model(data)
    print("Inference Speed (s): {:.4f}".format(time.time() - t0))

    return output

Use a pretrained resnet50 model from torchvision for example. Set to eval to change layer behavior to inference because we only want to use it for inference

In [ ]:
import torchvision
import torch

In [ ]:
model = torchvision.models.resnet50(pretrained=True).cuda().eval()

model_path = '/content/drive/MyDrive/JetBot/Day 4/Examples/resnet50.pth'
# model_path = 'resnet50.pth' # Use this on Jetbot
torch.save(model.state_dict(), model_path)
get_size(model_path)

In [ ]:
for name, param in model.state_dict().items():
    print("Layer: ", name)
    print(param.type())
    break

In [ ]:
# model = model.half()

# for name, param in model.state_dict().items():
#     print("Layer: ", name)
#     print(param.type())
#     break

Next, create some sample input that will be used to infer the shape and data types of our TensorRT engine

Make sure to set the right shape for input according to the model input shape.

In [ ]:
data = torch.randn((1, 3, 224, 224)).cuda()

In [ ]:
# If you encounter ModuleNotFoundError: No module named 'torch2trt' on Google Colab, restart the session
from torch2trt import torch2trt

model_trt = torch2trt(model, [data], fp16_mode=True)

Perform inference using converted model

In [ ]:
output_trt = run_inference(model_trt, data)

In [ ]:
output = run_inference(model, data)

# Check predictions error of both model

In [ ]:
print('max error: %f' % float(torch.max(torch.abs(output - output_trt))))

### Check the size of trt model

In [ ]:
model_path = '/content/drive/MyDrive/JetBot/Day 4/Examples/resnet50_trt.pth'
# model_path = 'resnet50_trt.pth' # Use this on Jetbot

torch.save(model_trt.state_dict(), model_path)

get_size(model_path)

In [ ]:
67358703 > 102549933

# Load tensorrt model

In [ ]:
from torch2trt import TRTModule

model_trt = TRTModule()

model_path = '/content/drive/MyDrive/JetBot/Day 4/Examples/resnet50_trt.pth'
# model_path = 'resnet50_trt.pth' # Use this on Jetbot

model_trt.load_state_dict(torch.load(model_path))